In [ ]:
import os 
import numpy as np 
import pandas as pd 

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import regularizers, initializers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_dir = os.path.join("C:/Users/kzhan/Desktop/archive")
training_dir = os.path.join(data_dir + "/training")
testing_dir = os.path.join(data_dir + "/testing")

In [ ]:
batch_size, epochs = 64, 20
learning_rate = 0.01

# ------------------------------------------------------------------------- #
# Define a training_dataset using the image_dataset_from_directory function #
# ------------------------------------------------------------------------- #
def get_training_dataset(training_dir):
    training_ds = tf.keras.preprocessing.image_dataset_from_directory(training_dir,
    batch_size = batch_size, seed = 123, subset = "training", color_mode = 'rgb', shuffle = True,
    validation_split = 0.2)
    return training_ds

# --------------------------------------------------------------------------- #
# Define a validation_dataset using the image_dataset_from_directory function #
# --------------------------------------------------------------------------- #
def get_validation_dataset(training_dir):
    validation_ds = tf.keras.preprocessing.image_dataset_from_directory(training_dir,
    batch_size = batch_size, seed = 123, subset = "validation", validation_split = 0.2,
    color_mode = 'rgb', shuffle = False)
    return validation_ds

# *Optional*
# Define a testing_dataset #
def get_testing_dataset(testing_dir):
    testing_ds = tf.keras.preprocessing.image_dataset_from_directory(testing_dir,
    batch_size = batch_size, shuffle = False, color_mode='rgb')

training_ds = get_training_dataset(training_dir)
validation_ds = get_validation_dataset(training_dir)
testing_ds = get_testing_dataset(testing_dir)

Found 2870 files belonging to 4 classes.
Using 2296 files for training.
Found 2870 files belonging to 4 classes.
Using 574 files for validation.
Found 394 files belonging to 4 classes.


In [ ]:
# ------------------- #
# Define an AUTOTUNER #
# ------------------- #
def autotuner():
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    return AUTOTUNE

# ---------------------------------------------- #
# Cache, Shuffle, and Prefetch the training data #
# ---------------------------------------------- #
def dataset_prefetch_cache_shuffle(training_data):
    training_data = training_data.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
    return training_data

# -------------------------------------- #
# Cache and Prefetch the validation data #
# -------------------------------------- #
def cache_prefetch(validation_data):
    validation_data = validation_data.cache().prefetch(buffer_size = AUTOTUNE)
    return validation_data

AUTOTUNE = autotuner()
training_ds = dataset_prefetch_cache_shuffle(training_ds) #prefetch, cache, and shuffle your training data if you want
validation_ds = cache_prefetch(validation_ds) # DO NOT SHUFFLE VALIDATION DATA

# --------------- #
# Simplified Code #
# --------------- #
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# training_ds = training_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
# validation_ds = validation_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
#similar to the first neural network we developed we must define a normalization layer
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255) 

#Normalize + map our dataset using a lambda function
normalized_ds = training_ds.map(lambda x_train, y_train: (normalization_layer(x_train), y_train))

#Iterate through our dataset 
image_batch, labels_batch = next(iter(normalized_ds))